# PERTAMINA LUBRICANTS

In [ ]:
import pandas as pd
from time import sleep
import requests
import json

In [ ]:
data = pd.read_csv('/content/sample_quadbin.csv')
#df = data.rename(columns={'lat': 'latitude', 'lng': 'longitude', 'name': 'place'})
df = data.rename(columns={'id': 'place'})
df = df[['bottom_left_lat', 'bottom_left_lon', 'top_right_lat', 'top_right_lon', 'place']]

In [ ]:
df

,bottom_left_lat,bottom_left_lon,top_right_lat,top_right_lon,place
0,-6.337137,106.875,-6.315299,106.896973,JAKARTA TIMUR - 5254864071169146879
1,-6.358975,106.875,-6.337137,106.896973,JAKARTA TIMUR - 5254864071202701311


In [ ]:
# Define the API Key.
API_KEY = '#############################################'

# Define the endpoint URL for the Nearby Search (New) API.
endpoint_search_text = "https://places.googleapis.com/v1/places:searchText"

In [ ]:
def search_places(keyword, max_result, bottom_left_lat, bottom_left_lon, top_right_lat, top_right_lon, place_name, page_token=None):
    payload = {
        "textQuery": keyword,
        "maxResultCount": max_result,
        "locationRestriction": {
          "rectangle": {
            "low": {
              "latitude": bottom_left_lat,
              "longitude": bottom_left_lon
            },
            "high": {
              "latitude": top_right_lat,
              "longitude": top_right_lon
            }
          }
        }
    }

    if page_token:
        payload["pageToken"] = page_token

    response = requests.post(endpoint_search_text, headers=headers, json=payload)
    if response.status_code == 200:
        places_result = response.json()
        for place in places_result.get('places', []):
            place['origin_place'] = place_name
        return places_result
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

In [ ]:
def get_all_places(keyword, max_result, bottom_left_lat, bottom_left_lon, top_right_lat, top_right_lon, place_name):
    all_results = []
    next_page_token = None
    iteration = 1

    while True:
        print(f"Requesting page {iteration} for {place_name}")
        result = search_places(keyword, max_result, bottom_left_lat, bottom_left_lon, top_right_lat, top_right_lon, place_name, next_page_token)

        if not result:
            break

        places = result.get('places', [])
        all_results.extend(places)

        print(f"Received {len(places)} places for {place_name}")

        next_page_token = result.get('nextPageToken')

        if not next_page_token or len(places) == 0:
            print("No more results or next page token found for", place_name)
            break

        iteration += 1

    return all_results

## Bengkel Mobil

In [ ]:
# CONFIGURATION
keyword = 'bengkel mobil' # input keyword
max_result = 20

# Headers untuk permintaan API
params = [
    'places.id', 'places.accessibilityOptions', 'places.addressComponents', 'places.adrFormatAddress',
    'places.businessStatus', 'places.displayName', 'places.formattedAddress', 'places.googleMapsUri',
    'places.iconBackgroundColor', 'places.iconMaskBaseUri', 'places.location', 'places.photos',
    'places.plusCode', 'places.primaryType', 'places.primaryTypeDisplayName', 'places.shortFormattedAddress',
    'places.subDestinations', 'places.types', 'places.utcOffsetMinutes', 'places.viewport', 'places.currentOpeningHours',
    'places.currentSecondaryOpeningHours', 'places.internationalPhoneNumber', 'places.nationalPhoneNumber', 'places.priceLevel',
    'places.rating', 'places.regularOpeningHours', 'places.regularSecondaryOpeningHours', 'places.userRatingCount', 'places.websiteUri'
]

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': ','.join(params) + ',nextPageToken'
}

In [ ]:
# Loop melalui setiap baris dalam DataFrame dan panggil Text Search API
nearby_bengkel_mobil = []
for index, row in df.iterrows():
    bottom_left_lat = float(row['bottom_left_lat'])
    bottom_left_lon = float(row['bottom_left_lon'])
    top_right_lat = float(row['top_right_lat'])
    top_right_lon = float(row['top_right_lon'])
    place_name = row['place']
    places = get_all_places(keyword, max_result, bottom_left_lat, bottom_left_lon, top_right_lat, top_right_lon, place_name)
    nearby_bengkel_mobil.extend(places)

Requesting page 1 for JAKARTA TIMUR - 5254864071169146879
Received 20 places for JAKARTA TIMUR - 5254864071169146879
Requesting page 2 for JAKARTA TIMUR - 5254864071169146879
Received 20 places for JAKARTA TIMUR - 5254864071169146879
Requesting page 3 for JAKARTA TIMUR - 5254864071169146879
Received 20 places for JAKARTA TIMUR - 5254864071169146879
No more results or next page token found for JAKARTA TIMUR - 5254864071169146879
Requesting page 1 for JAKARTA TIMUR - 5254864071202701311
Received 20 places for JAKARTA TIMUR - 5254864071202701311
Requesting page 2 for JAKARTA TIMUR - 5254864071202701311
Received 20 places for JAKARTA TIMUR - 5254864071202701311
Requesting page 3 for JAKARTA TIMUR - 5254864071202701311
Received 20 places for JAKARTA TIMUR - 5254864071202701311
No more results or next page token found for JAKARTA TIMUR - 5254864071202701311


In [ ]:
# Saving JSON
with open('nearby_bengkel_mobil.json', 'w') as f: # Change name as POI
    json.dump(nearby_bengkel_mobil, f, indent=4)  # Change name as POI

In [ ]:
def extract_POI(data):
    extract_bengkel_mobil = [] # Change name as POI
    for item in data:
        extract = {}
        extract['name'] = item.get('displayName', {}).get('text', '')
        extract['primary_type'] = item.get('primaryType', '')
        extract['latitude'] = item.get('location', {}).get('latitude', '')
        extract['longitude'] = item.get('location', {}).get('longitude', '')
        extract['address'] = item.get('formattedAddress', '')
        extract['business_status'] = item.get('businessStatus', '')
        extract['types'] = item.get('types', '')
        extract['price_level'] = item.get('priceLevel', '')
        extract['rating'] = item.get('rating', '')
        extract['reviews'] = item.get('reviews', '')
        extract['user_ratings_total'] = item.get('userRatingCount', '')
        extract['operational_hours'] = item.get('regularOpeningHours', {}).get('periods', '')
        extract['phone_number'] = item.get('nationalPhoneNumber', '')
        extract['place_id'] = item.get('id', '')
        extract['website_url'] = item.get('websiteUri', '')
        extract['gmaps_url'] = item.get('googleMapsUri', '')
        extract['origin_place'] = item.get('origin_place', '')
        # extract['wheelchair_parking'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleParking', '')
        # extract['wheelchair_entrance'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleEntrance', '')
        # extract['wheelchair_restroom'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleRestroom', '')
        # extract['wheelchair_seating'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleSeating', '')
        # extract['allow_dogs'] = item.get('allowsDogs', '')
        # extract['curbside_pickup'] = item.get('curbsidePickup', '')
        # extract['delivery'] = item.get('delivery', '')
        # extract['dine_in'] = item.get('dineIn', '')
        # extract['good_for_children'] = item.get('goodForChildren', '')
        # extract['good_for_groups'] = item.get('goodForGroups', '')
        # extract['good_for_watching_sports'] = item.get('goodForWatchingSports', '')
        # extract['live_music'] = item.get('liveMusic', '')
        # extract['menu_for_children'] = item.get('menuForChildren', '')
        # extract['paid_parking_lot'] = item.get('parkingOptions', {}).get('paidParkingLot', '')
        # extract['paid_street_parking'] = item.get('parkingOptions', {}).get('paidStreetParking', '')
        # extract['valet_parking'] = item.get('parkingOptions', {}).get('valetParking', '')
        # extract['credit_card_payment'] = item.get('paymentOptions', {}).get('acceptsCreditCards', '')
        # extract['debit_card_payment'] = item.get('paymentOptions', {}).get('acceptsDebitCards', '')
        # extract['only_cash_payment'] = item.get('paymentOptions', {}).get('acceptsCashOnly', '')
        # extract['nfc_payment'] = item.get('paymentOptions', {}).get('acceptsNfc', '')
        # extract['online_payment'] = item.get('paymentOptions', {}).get('acceptsOnlinePayments', '')
        # extract['outdoor_seating'] = item.get('outdoorSeating', '')
        # extract['reservable'] = item.get('reservable', '')
        # extract['restroom'] = item.get('restroom', '')
        # extract['serves_beer'] = item.get('servesBeer', '')
        # extract['serves_breakfast'] = item.get('servesBreakfast', '')
        # extract['serves_brunch'] = item.get('servesBrunch', '')
        # extract['serves_cocktails'] = item.get('servesCocktails', '')
        # extract['serves_coffee'] = item.get('servesCoffee', '')
        # extract['serves_dessert'] = item.get('servesDessert', '')
        # extract['serves_dinner'] = item.get('servesDinner', '')
        # extract['serves_lunch'] = item.get('servesLunch', '')
        # extract['serves_vegetarian_food'] = item.get('servesVegetarianFood', '')
        # extract['serves_wine'] = item.get('servesWine', '')
        # extract['takeout'] = item.get('takeout', '')
        extract_bengkel_mobil.append(extract)  # Change name as POI
    return extract_bengkel_mobil  # Change name as POI

# Flatten the JSON data
extract_bengkel_mobil = extract_POI(nearby_bengkel_mobil) # change name as POI

# Convert the flattened data to a DataFrame
POI_bengkel_mobil = pd.DataFrame(extract_bengkel_mobil) # Change name as POI
POI_bengkel_mobil.to_csv('POI_bengkel_mobil.csv', index=False)

In [ ]:
POI_bengkel_mobil

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place
0,Bengkel BONPIS,car_repair,-6.336377,106.890371,"jl.KADIH No.1, RT.1/RW.1, Cipayung, Kec. Cipay...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.5,,16,"[{'open': {'day': 0, 'hour': 0, 'minute': 0}}]",0813-2967-3358,ChIJJ5yla87taS4RzEa_Qm5c0pM,,https://maps.google.com/?cid=10651677697320634060,JAKARTA TIMUR - 5254864071169146879
1,Green Nitrogen Cipayung JakTim SPBU 3413808,car_repair,-6.332083,106.888781,"Jl. Cipayung Raya No.31 16, RT.16/RW.1, Cipayu...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.5,,11,"[{'open': {'day': 0, 'hour': 0, 'minute': 0}}]",(021) 29286176,ChIJ84NBCCLtaS4RMZvGO55RC28,http://www.green-nitrogen.com/,https://maps.google.com/?cid=8001578903003372337,JAKARTA TIMUR - 5254864071169146879
2,Mekanik panggilan Bengkel mobil mustakim,car_repair,-6.322214,106.879017,"Bengkel mustakim, Jl. Mebel No.37 13, RT.13/RW...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4,,5,"[{'open': {'day': 0, 'hour': 0, 'minute': 0}}]",0815-9537-172,ChIJ__GOwp7taS4RZDB2BzjNPJc,,https://maps.google.com/?cid=10897810838856806500,JAKARTA TIMUR - 5254864071169146879
3,OLE DINAMO (BENGKEL DINAMO),car_repair,-6.323571,106.888340,"MVGQ+H8G, RT.2/RW.6, Cipayung, East Jakarta Ci...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,12,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",0896-2119-8474,ChIJD7LnipftaS4R_EyTd-VACl0,,https://maps.google.com/?cid=6704242349592235260,JAKARTA TIMUR - 5254864071169146879
4,Roby Motor,car_repair,-6.327066,106.882582,"10, Jl. Penganten Ali IX No.26, RT.6/RW.6, Cir...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.4,,88,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",0812-8408-4379,ChIJF0p2yRDtaS4RpG10J6oB7Dg,,https://maps.google.com/?cid=4101655190945688996,JAKARTA TIMUR - 5254864071169146879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Bengkel Cat Mobil Miftah Motor,car_repair,-6.342059,106.878736,"Jl. Manunggal Raya No.19, RT.13/RW.8, Klp. Dua...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,3,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0813-1936-9890,ChIJj31rCBztaS4R3Q302zIr_Vs,,https://maps.google.com/?cid=6628501723997801949,JAKARTA TIMUR - 5254864071202701311
116,Bengkel Horas Motor,car_repair,-6.344425,106.883339,"MV4M+68P, Jl. Raya Klp. Dua Wetan, RT.8/RW.5, ...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,3,,,ChIJVbwGjODtaS4RSxFpIeBMhmI,,https://maps.google.com/?cid=7099446388113084747,JAKARTA TIMUR - 5254864071202701311
117,faozan sparepart mobil,auto_parts_store,-6.342465,106.891619,"sparepart mobil samping, Gg. H. Acep Jl. Malak...",OPERATIONAL,"[auto_parts_store, car_repair, point_of_intere...",,4.9,,7,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0812-8577-7323,ChIJkxmX0N3saS4Rxa7PEeodies,,https://maps.google.com/?cid=16972129561880800965,JAKARTA TIMUR - 5254864071202701311
118,BABEH PER,auto_parts_store,-6.351641,106.879287,"Jl. Lap. Tembak No.30 7, RT.7/RW.4, Cibubur, K...",OPERATIONAL,"[auto_parts_store, car_repair, point_of_intere...",,4.1,,20,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0859-3047-1420,ChIJ6Yw0ktvtaS4RvJVZ1Ra82vM,,https://maps.google.com/?cid=17571563702394131900,JAKARTA TIMUR - 5254864071202701311


## Bengkel Motor

In [ ]:
# CONFIGURATION
radius = 3000 # in meters
keyword = 'bengkel motor' # input keyword
max_result = 20

# Headers untuk permintaan API
params = [
    'places.id', 'places.accessibilityOptions', 'places.addressComponents', 'places.adrFormatAddress',
    'places.businessStatus', 'places.displayName', 'places.formattedAddress', 'places.googleMapsUri',
    'places.iconBackgroundColor', 'places.iconMaskBaseUri', 'places.location', 'places.photos',
    'places.plusCode', 'places.primaryType', 'places.primaryTypeDisplayName', 'places.shortFormattedAddress',
    'places.subDestinations', 'places.types', 'places.utcOffsetMinutes', 'places.viewport', 'places.currentOpeningHours',
    'places.currentSecondaryOpeningHours', 'places.internationalPhoneNumber', 'places.nationalPhoneNumber', 'places.priceLevel',
    'places.rating', 'places.regularOpeningHours', 'places.regularSecondaryOpeningHours', 'places.userRatingCount', 'places.websiteUri'
]

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': ','.join(params) + ',nextPageToken'
}

In [ ]:
# Loop melalui setiap baris dalam DataFrame dan panggil Text Search API
nearby_bengkel_motor = []
for index, row in df.iterrows():
    latitude = float(row['latitude'])
    longitude = float(row['longitude'])
    place_name = row['place']
    places = get_all_places(keyword, max_result, latitude, longitude, place_name)
    nearby_bengkel_motor.extend(places)

Requesting page 1 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 2 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 3 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
No more results or next page token found for TANJUNG PRIOK
Requesting page 1 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 2 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 3 for PADEMANGAN
Received 20 places for PADEMANGAN
No more results or next page token found for PADEMANGAN
Requesting page 1 for CILINCING
Received 20 places for CILINCING
Requesting page 2 for CILINCING
Received 20 places for CILINCING
Requesting page 3 for CILINCING
Received 20 places for CILINCING
No more results or next page token found for CILINCING
Requesting page 1 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 2 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 3 for KELAPA GADING
Received 20 places for KELAPA GAD

In [ ]:
# Saving JSON
with open('nearby_bengkel_motor.json', 'w') as f: # Change name as POI
    json.dump(nearby_bengkel_motor, f, indent=4)  # Change name as POI

In [ ]:
def extract_POI(data):
    extract_bengkel_motor = [] # Change name as POI
    for item in data:
        extract = {}
        extract['name'] = item.get('displayName', {}).get('text', '')
        extract['primary_type'] = item.get('primaryType', '')
        extract['latitude'] = item.get('location', {}).get('latitude', '')
        extract['longitude'] = item.get('location', {}).get('longitude', '')
        extract['address'] = item.get('formattedAddress', '')
        extract['business_status'] = item.get('businessStatus', '')
        extract['types'] = item.get('types', '')
        extract['price_level'] = item.get('priceLevel', '')
        extract['rating'] = item.get('rating', '')
        extract['reviews'] = item.get('reviews', '')
        extract['user_ratings_total'] = item.get('userRatingCount', '')
        extract['operational_hours'] = item.get('regularOpeningHours', {}).get('periods', '')
        extract['phone_number'] = item.get('nationalPhoneNumber', '')
        extract['place_id'] = item.get('id', '')
        extract['website_url'] = item.get('websiteUri', '')
        extract['gmaps_url'] = item.get('googleMapsUri', '')
        extract['origin_place'] = item.get('origin_place', '')
        # extract['wheelchair_parking'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleParking', '')
        # extract['wheelchair_entrance'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleEntrance', '')
        # extract['wheelchair_restroom'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleRestroom', '')
        # extract['wheelchair_seating'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleSeating', '')
        # extract['allow_dogs'] = item.get('allowsDogs', '')
        # extract['curbside_pickup'] = item.get('curbsidePickup', '')
        # extract['delivery'] = item.get('delivery', '')
        # extract['dine_in'] = item.get('dineIn', '')
        # extract['good_for_children'] = item.get('goodForChildren', '')
        # extract['good_for_groups'] = item.get('goodForGroups', '')
        # extract['good_for_watching_sports'] = item.get('goodForWatchingSports', '')
        # extract['live_music'] = item.get('liveMusic', '')
        # extract['menu_for_children'] = item.get('menuForChildren', '')
        # extract['paid_parking_lot'] = item.get('parkingOptions', {}).get('paidParkingLot', '')
        # extract['paid_street_parking'] = item.get('parkingOptions', {}).get('paidStreetParking', '')
        # extract['valet_parking'] = item.get('parkingOptions', {}).get('valetParking', '')
        # extract['credit_card_payment'] = item.get('paymentOptions', {}).get('acceptsCreditCards', '')
        # extract['debit_card_payment'] = item.get('paymentOptions', {}).get('acceptsDebitCards', '')
        # extract['only_cash_payment'] = item.get('paymentOptions', {}).get('acceptsCashOnly', '')
        # extract['nfc_payment'] = item.get('paymentOptions', {}).get('acceptsNfc', '')
        # extract['online_payment'] = item.get('paymentOptions', {}).get('acceptsOnlinePayments', '')
        # extract['outdoor_seating'] = item.get('outdoorSeating', '')
        # extract['reservable'] = item.get('reservable', '')
        # extract['restroom'] = item.get('restroom', '')
        # extract['serves_beer'] = item.get('servesBeer', '')
        # extract['serves_breakfast'] = item.get('servesBreakfast', '')
        # extract['serves_brunch'] = item.get('servesBrunch', '')
        # extract['serves_cocktails'] = item.get('servesCocktails', '')
        # extract['serves_coffee'] = item.get('servesCoffee', '')
        # extract['serves_dessert'] = item.get('servesDessert', '')
        # extract['serves_dinner'] = item.get('servesDinner', '')
        # extract['serves_lunch'] = item.get('servesLunch', '')
        # extract['serves_vegetarian_food'] = item.get('servesVegetarianFood', '')
        # extract['serves_wine'] = item.get('servesWine', '')
        # extract['takeout'] = item.get('takeout', '')
        extract_bengkel_motor.append(extract)  # Change name as POI
    return extract_bengkel_motor  # Change name as POI

# Flatten the JSON data
extract_bengkel_motor = extract_POI(nearby_bengkel_motor) # change name as POI

# Convert the flattened data to a DataFrame
POI_bengkel_motor = pd.DataFrame(extract_bengkel_motor) # Change name as POI
POI_bengkel_motor.to_csv('POI_bengkel_motor.csv', index=False)

In [ ]:
POI_bengkel_motor

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place
0,Bengkel Motor Rx King,car_repair,-6.137121,106.880134,Bengkel Karya Jaya Motor/KJM Sport. Jl. Agung ...,OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.7,,34,"[{'open': {'day': 0, 'hour': 6, 'minute': 0}, ...",0812-1099-9467,ChIJCc_6o2P1aS4RMbu6K9b8pDs,https://www.facebook.com/sovian.sport?mibextid...,https://maps.google.com/?cid=4297837941205154609,TANJUNG PRIOK
1,Bengkel Motor Honda Abadi Makmur (Prog.Vokasi),car_repair,-6.137504,106.873324,"Jl. Danau Sunter Utara No.20 Blok.J12, Sunter ...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.7,,107,"[{'open': {'day': 1, 'hour': 11, 'minute': 0},...",0857-8261-9868,ChIJOfKy6V_1aS4RDdy1Nta3FLU,http://www.astra-honda.com/,https://maps.google.com/?cid=13048256151067941901,TANJUNG PRIOK
2,Lambang Utama Motor,car_repair,-6.125434,106.883231,"Jalan Warakas Raya, No. 78, Warakas, luar samp...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.9,,42,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",0895-3331-69331,ChIJge7-rlwfai4Rz1CrQ8e46EM,,https://maps.google.com/?cid=4893364161111347407,TANJUNG PRIOK
3,Bengkel motor anugrah kembar,car_repair,-6.124289,106.872733,"Kode Pos:14370, Jl. Gg. 21 No.50D, RT./RW:11/R...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.8,,4,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0852-9007-1904,ChIJpdMFKYEfai4RgxMO0xYvVeU,,https://maps.google.com/?cid=16525166182782604163,TANJUNG PRIOK
4,Heru Motor,car_repair,-6.126063,106.883966,"Jl. Papanggo I No.207, RT.6/RW.10, Sungai Bamb...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,12,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0813-1473-8858,ChIJ3S7YYdwfai4RFkORI37fbkk,,https://maps.google.com/?cid=5291412345062507286,TANJUNG PRIOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Bengkel Motor Kasturi,car_repair,-6.134699,106.739996,"VP8Q+4XH, RT.22/RW.16, Kapuk, Cengkareng, West...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4,,2,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",,ChIJZfyXPlkdai4RWv0ADg_xQKY,,https://maps.google.com/?cid=11979840055767268698,PENJARINGAN
416,Enduro Motor,car_repair,-6.130541,106.720658,"Jl. Dharma Bakti No.9 9, RT.9/RW.10, Cengkaren...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,7,"[{'open': {'day': 6, 'hour': 0, 'minute': 0}, ...",,ChIJd1kPiXMDai4R-dKM5OW9yqE,,https://maps.google.com/?cid=11658339380481544953,PENJARINGAN
417,BENGKEL MOTOR IDUN,car_repair,-6.131479,106.719950,"Jl. Pulo Harapan Indah No.147 8, RT.8/RW.10, C...",OPERATIONAL,"[electronics_store, car_repair, store, point_o...",,3,,3,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0896-7115-1844,ChIJo_VjX6wCai4R3jmvWKSQ6yk,,https://maps.google.com/?cid=3020667010618964446,PENJARINGAN
418,Bengkel Bajaj Mediamotor,car_repair,-6.136121,106.747123,"Jl. Kapuk Santri No.81 137, RT.12/RW.12, Kapuk...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.4,,7,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0857-1020-2181,ChIJaRIm1l8dai4RWShZ5uTjyuA,,https://maps.google.com/?cid=16198009582048061529,PENJARINGAN


## Dealer

In [ ]:
# CONFIGURATION
radius = 3000 # in meters
keyword = 'dealer' # input keyword
max_result = 20

# Headers untuk permintaan API
params = [
    'places.id', 'places.accessibilityOptions', 'places.addressComponents', 'places.adrFormatAddress',
    'places.businessStatus', 'places.displayName', 'places.formattedAddress', 'places.googleMapsUri',
    'places.iconBackgroundColor', 'places.iconMaskBaseUri', 'places.location', 'places.photos',
    'places.plusCode', 'places.primaryType', 'places.primaryTypeDisplayName', 'places.shortFormattedAddress',
    'places.subDestinations', 'places.types', 'places.utcOffsetMinutes', 'places.viewport', 'places.currentOpeningHours',
    'places.currentSecondaryOpeningHours', 'places.internationalPhoneNumber', 'places.nationalPhoneNumber', 'places.priceLevel',
    'places.rating', 'places.regularOpeningHours', 'places.regularSecondaryOpeningHours', 'places.userRatingCount', 'places.websiteUri'
]

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': ','.join(params) + ',nextPageToken'
}

In [ ]:
# Loop melalui setiap baris dalam DataFrame dan panggil Text Search API
nearby_dealer = []
for index, row in df.iterrows():
    latitude = float(row['latitude'])
    longitude = float(row['longitude'])
    place_name = row['place']
    places = get_all_places(keyword, max_result, latitude, longitude, place_name)
    nearby_dealer.extend(places)

Requesting page 1 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 2 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 3 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
No more results or next page token found for TANJUNG PRIOK
Requesting page 1 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 2 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 3 for PADEMANGAN
Received 20 places for PADEMANGAN
No more results or next page token found for PADEMANGAN
Requesting page 1 for CILINCING
Received 20 places for CILINCING
Requesting page 2 for CILINCING
Received 20 places for CILINCING
Requesting page 3 for CILINCING
Received 20 places for CILINCING
No more results or next page token found for CILINCING
Requesting page 1 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 2 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 3 for KELAPA GADING
Received 20 places for KELAPA GAD

In [ ]:
# Saving JSON
with open('nearby_dealer.json', 'w') as f: # Change name as POI
    json.dump(nearby_dealer, f, indent=4)  # Change name as POI

In [ ]:
def extract_POI(data):
    extract_dealer = [] # Change name as POI
    for item in data:
        extract = {}
        extract['name'] = item.get('displayName', {}).get('text', '')
        extract['primary_type'] = item.get('primaryType', '')
        extract['latitude'] = item.get('location', {}).get('latitude', '')
        extract['longitude'] = item.get('location', {}).get('longitude', '')
        extract['address'] = item.get('formattedAddress', '')
        extract['business_status'] = item.get('businessStatus', '')
        extract['types'] = item.get('types', '')
        extract['price_level'] = item.get('priceLevel', '')
        extract['rating'] = item.get('rating', '')
        extract['reviews'] = item.get('reviews', '')
        extract['user_ratings_total'] = item.get('userRatingCount', '')
        extract['operational_hours'] = item.get('regularOpeningHours', {}).get('periods', '')
        extract['phone_number'] = item.get('nationalPhoneNumber', '')
        extract['place_id'] = item.get('id', '')
        extract['website_url'] = item.get('websiteUri', '')
        extract['gmaps_url'] = item.get('googleMapsUri', '')
        extract['origin_place'] = item.get('origin_place', '')
        # extract['wheelchair_parking'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleParking', '')
        # extract['wheelchair_entrance'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleEntrance', '')
        # extract['wheelchair_restroom'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleRestroom', '')
        # extract['wheelchair_seating'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleSeating', '')
        # extract['allow_dogs'] = item.get('allowsDogs', '')
        # extract['curbside_pickup'] = item.get('curbsidePickup', '')
        # extract['delivery'] = item.get('delivery', '')
        # extract['dine_in'] = item.get('dineIn', '')
        # extract['good_for_children'] = item.get('goodForChildren', '')
        # extract['good_for_groups'] = item.get('goodForGroups', '')
        # extract['good_for_watching_sports'] = item.get('goodForWatchingSports', '')
        # extract['live_music'] = item.get('liveMusic', '')
        # extract['menu_for_children'] = item.get('menuForChildren', '')
        # extract['paid_parking_lot'] = item.get('parkingOptions', {}).get('paidParkingLot', '')
        # extract['paid_street_parking'] = item.get('parkingOptions', {}).get('paidStreetParking', '')
        # extract['valet_parking'] = item.get('parkingOptions', {}).get('valetParking', '')
        # extract['credit_card_payment'] = item.get('paymentOptions', {}).get('acceptsCreditCards', '')
        # extract['debit_card_payment'] = item.get('paymentOptions', {}).get('acceptsDebitCards', '')
        # extract['only_cash_payment'] = item.get('paymentOptions', {}).get('acceptsCashOnly', '')
        # extract['nfc_payment'] = item.get('paymentOptions', {}).get('acceptsNfc', '')
        # extract['online_payment'] = item.get('paymentOptions', {}).get('acceptsOnlinePayments', '')
        # extract['outdoor_seating'] = item.get('outdoorSeating', '')
        # extract['reservable'] = item.get('reservable', '')
        # extract['restroom'] = item.get('restroom', '')
        # extract['serves_beer'] = item.get('servesBeer', '')
        # extract['serves_breakfast'] = item.get('servesBreakfast', '')
        # extract['serves_brunch'] = item.get('servesBrunch', '')
        # extract['serves_cocktails'] = item.get('servesCocktails', '')
        # extract['serves_coffee'] = item.get('servesCoffee', '')
        # extract['serves_dessert'] = item.get('servesDessert', '')
        # extract['serves_dinner'] = item.get('servesDinner', '')
        # extract['serves_lunch'] = item.get('servesLunch', '')
        # extract['serves_vegetarian_food'] = item.get('servesVegetarianFood', '')
        # extract['serves_wine'] = item.get('servesWine', '')
        # extract['takeout'] = item.get('takeout', '')
        extract_dealer.append(extract)  # Change name as POI
    return extract_dealer  # Change name as POI

# Flatten the JSON data
extract_dealer = extract_POI(nearby_dealer) # change name as POI

# Convert the flattened data to a DataFrame
POI_dealer = pd.DataFrame(extract_dealer) # Change name as POI
POI_dealer.to_csv('POI_dealer.csv', index=False)

In [ ]:
POI_dealer

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place
0,Marketing Mitsubishi Sunter,car_dealer,-6.137999,106.876823,"Jl. Danau Sunter Utara No.14, Papanggo, Kec. T...",OPERATIONAL,"[car_dealer, store, point_of_interest, establi...",,4.9,,118,"[{'open': {'day': 0, 'hour': 0, 'minute': 0}}]",0877-8801-1122,ChIJ66a7Fnf1aS4RZeuWCpUs_BY,http://bit.ly/mitsubishisunterjakarta,https://maps.google.com/?cid=1656247781604911973,TANJUNG PRIOK
1,Hyundai Sunter Official,car_dealer,-6.131189,106.855485,Jl. Danau Sunter Barat Sunter No.7 Blok A1 – A...,OPERATIONAL,"[car_dealer, store, point_of_interest, establi...",,4.5,,107,"[{'open': {'day': 0, 'hour': 8, 'minute': 30},...",(021) 6518565,ChIJA-Sr8Dgfai4Rrg_nM68Pago,http://www.hyundai-assistant.com/dealer-profil...,https://maps.google.com/?cid=750429533075083182,TANJUNG PRIOK
2,Wuling Sunter,car_dealer,-6.138043,106.864719,"Kompleks Sunter Garden blok D1 kav 12-13 no 8,...",OPERATIONAL,"[car_dealer, store, point_of_interest, establi...",,4.8,,138,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",(021) 22657777,ChIJ_WIR43n1aS4RgQuq8qSRmjQ,,https://maps.google.com/?cid=3790502174017588097,TANJUNG PRIOK
3,Toyota Auto2000 Yos Sudarso,car_dealer,-6.139669,106.891024,"Jl. Yos Sudarso Blok A1-8 No.Kav.46-48, RT.10/...",OPERATIONAL,"[car_dealer, store, point_of_interest, establi...",,4.7,,2782,"[{'open': {'day': 1, 'hour': 7, 'minute': 0}, ...",(021) 65302000,ChIJxcMdc131aS4RLhuwD4Lud9o,https://auto2000.co.id/cabang/auto2000yossudar...,https://maps.google.com/?cid=15742313264873216814,TANJUNG PRIOK
4,Dealer Mitsubishi Sunter - Jakarta Utara,car_dealer,-6.137863,106.876834,"Jl. Danau Sunter Utara No.8, RT.6/RW.14, Sunte...",OPERATIONAL,"[car_dealer, store, point_of_interest, establi...",,4.8,,19,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",0813-1701-1781,ChIJSZSYDon1aS4RXohRtgLTL5k,,https://maps.google.com/?cid=11038273220310567006,TANJUNG PRIOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,MG Motor Pluit - Gema Auto Citra,car_dealer,-6.120882,106.791494,"Jl. Pluit Putra No.15, Pluit, Kec. Penjaringan...",OPERATIONAL,"[car_dealer, point_of_interest, store, establi...",,4.4,,11,"[{'open': {'day': 0, 'hour': 8, 'minute': 30},...",(021) 6693673,ChIJG1Q7uz4pEawRPOWbsgWWICo,,https://maps.google.com/?cid=3035591100063278396,PENJARINGAN
416,Mitsubishi Pantai Indah Kapuk,car_dealer,-6.092499,106.745535,JL. GOLF ISLAND BOULEVARD LOT K NO.01 RT.10/RW...,OPERATIONAL,"[car_dealer, point_of_interest, store, establi...",,5,,4,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",0813-8286-8993,ChIJ3R8QAUD1aS4ROXHuk64v9UQ,https://www.mitsubishipik.com/,https://maps.google.com/?cid=4968930190725902649,PENJARINGAN
417,Nusantara Motor,car_dealer,-6.106270,106.711078,"Jl. Kamal Raya No.12 13, RT.15/RW.9, Tegal Alu...",OPERATIONAL,"[car_dealer, point_of_interest, store, establi...",,5,,1,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0812-1998-9267,ChIJ2flHhtoCai4R4xA0W6dUHiU,,https://maps.google.com/?cid=2674668306471063779,PENJARINGAN
418,Hyundai Pantai Indah Kapuk,car_dealer,-6.109277,106.740166,"Jl. Pantai Indah Kapuk, Kamal Muara, Kec. Penj...",OPERATIONAL,"[car_dealer, point_of_interest, store, establi...",,1.8,,5,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",0821-2929-0006,ChIJJWdw_fsdai4RYeTxt2-hNow,https://www.hyundaim2.com/,https://maps.google.com/?cid=10103440315251811425,PENJARINGAN


## Bengkel Tap Oli

In [ ]:
# CONFIGURATION
radius = 3000 # in meters
keyword = 'oil change' # input keyword
max_result = 20

# Headers untuk permintaan API
params = [
    'places.id', 'places.accessibilityOptions', 'places.addressComponents', 'places.adrFormatAddress',
    'places.businessStatus', 'places.displayName', 'places.formattedAddress', 'places.googleMapsUri',
    'places.iconBackgroundColor', 'places.iconMaskBaseUri', 'places.location', 'places.photos',
    'places.plusCode', 'places.primaryType', 'places.primaryTypeDisplayName', 'places.shortFormattedAddress',
    'places.subDestinations', 'places.types', 'places.utcOffsetMinutes', 'places.viewport', 'places.currentOpeningHours',
    'places.currentSecondaryOpeningHours', 'places.internationalPhoneNumber', 'places.nationalPhoneNumber', 'places.priceLevel',
    'places.rating', 'places.regularOpeningHours', 'places.regularSecondaryOpeningHours', 'places.userRatingCount', 'places.websiteUri'
]

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': ','.join(params) + ',nextPageToken'
}

In [ ]:
# Loop melalui setiap baris dalam DataFrame dan panggil Text Search API
nearby_bengkel_oli = []
for index, row in df.iterrows():
    latitude = float(row['latitude'])
    longitude = float(row['longitude'])
    place_name = row['place']
    places = get_all_places(keyword, max_result, latitude, longitude, place_name)
    nearby_bengkel_oli.extend(places)

Requesting page 1 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 2 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 3 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
No more results or next page token found for TANJUNG PRIOK
Requesting page 1 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 2 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 3 for PADEMANGAN
Received 20 places for PADEMANGAN
No more results or next page token found for PADEMANGAN
Requesting page 1 for CILINCING
Received 20 places for CILINCING
Requesting page 2 for CILINCING
Received 20 places for CILINCING
Requesting page 3 for CILINCING
Received 20 places for CILINCING
No more results or next page token found for CILINCING
Requesting page 1 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 2 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 3 for KELAPA GADING
Received 20 places for KELAPA GAD

In [ ]:
# Saving JSON
with open('nearby_bengkel_oli.json', 'w') as f: # Change name as POI
    json.dump(nearby_bengkel_oli, f, indent=4)  # Change name as POI

In [ ]:
def extract_POI(data):
    extract_bengkel_oli = [] # Change name as POI
    for item in data:
        extract = {}
        extract['name'] = item.get('displayName', {}).get('text', '')
        extract['primary_type'] = item.get('primaryType', '')
        extract['latitude'] = item.get('location', {}).get('latitude', '')
        extract['longitude'] = item.get('location', {}).get('longitude', '')
        extract['address'] = item.get('formattedAddress', '')
        extract['business_status'] = item.get('businessStatus', '')
        extract['types'] = item.get('types', '')
        extract['price_level'] = item.get('priceLevel', '')
        extract['rating'] = item.get('rating', '')
        extract['reviews'] = item.get('reviews', '')
        extract['user_ratings_total'] = item.get('userRatingCount', '')
        extract['operational_hours'] = item.get('regularOpeningHours', {}).get('periods', '')
        extract['phone_number'] = item.get('nationalPhoneNumber', '')
        extract['place_id'] = item.get('id', '')
        extract['website_url'] = item.get('websiteUri', '')
        extract['gmaps_url'] = item.get('googleMapsUri', '')
        extract['origin_place'] = item.get('origin_place', '')
        # extract['wheelchair_parking'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleParking', '')
        # extract['wheelchair_entrance'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleEntrance', '')
        # extract['wheelchair_restroom'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleRestroom', '')
        # extract['wheelchair_seating'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleSeating', '')
        # extract['allow_dogs'] = item.get('allowsDogs', '')
        # extract['curbside_pickup'] = item.get('curbsidePickup', '')
        # extract['delivery'] = item.get('delivery', '')
        # extract['dine_in'] = item.get('dineIn', '')
        # extract['good_for_children'] = item.get('goodForChildren', '')
        # extract['good_for_groups'] = item.get('goodForGroups', '')
        # extract['good_for_watching_sports'] = item.get('goodForWatchingSports', '')
        # extract['live_music'] = item.get('liveMusic', '')
        # extract['menu_for_children'] = item.get('menuForChildren', '')
        # extract['paid_parking_lot'] = item.get('parkingOptions', {}).get('paidParkingLot', '')
        # extract['paid_street_parking'] = item.get('parkingOptions', {}).get('paidStreetParking', '')
        # extract['valet_parking'] = item.get('parkingOptions', {}).get('valetParking', '')
        # extract['credit_card_payment'] = item.get('paymentOptions', {}).get('acceptsCreditCards', '')
        # extract['debit_card_payment'] = item.get('paymentOptions', {}).get('acceptsDebitCards', '')
        # extract['only_cash_payment'] = item.get('paymentOptions', {}).get('acceptsCashOnly', '')
        # extract['nfc_payment'] = item.get('paymentOptions', {}).get('acceptsNfc', '')
        # extract['online_payment'] = item.get('paymentOptions', {}).get('acceptsOnlinePayments', '')
        # extract['outdoor_seating'] = item.get('outdoorSeating', '')
        # extract['reservable'] = item.get('reservable', '')
        # extract['restroom'] = item.get('restroom', '')
        # extract['serves_beer'] = item.get('servesBeer', '')
        # extract['serves_breakfast'] = item.get('servesBreakfast', '')
        # extract['serves_brunch'] = item.get('servesBrunch', '')
        # extract['serves_cocktails'] = item.get('servesCocktails', '')
        # extract['serves_coffee'] = item.get('servesCoffee', '')
        # extract['serves_dessert'] = item.get('servesDessert', '')
        # extract['serves_dinner'] = item.get('servesDinner', '')
        # extract['serves_lunch'] = item.get('servesLunch', '')
        # extract['serves_vegetarian_food'] = item.get('servesVegetarianFood', '')
        # extract['serves_wine'] = item.get('servesWine', '')
        # extract['takeout'] = item.get('takeout', '')
        extract_bengkel_oli.append(extract)  # Change name as POI
    return extract_bengkel_oli  # Change name as POI

# Flatten the JSON data
extract_bengkel_oli = extract_POI(nearby_bengkel_oli) # change name as POI

# Convert the flattened data to a DataFrame
POI_bengkel_oli = pd.DataFrame(extract_bengkel_oli) # Change name as POI
POI_bengkel_oli.to_csv('POI_bengkel_oli.csv', index=False)

In [ ]:
POI_bengkel_oli

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place
0,Castrol Auto Service - Auto Lube - Redhouse Lu...,car_repair,-6.149707,106.881394,"Jl. Danau Indah Barat No.12 Blok C3 No, RT.1/R...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.6,,34,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",(021) 65303212,ChIJYXn4tG31aS4RpfFAMoxdFhY,,https://maps.google.com/?cid=1591562375041970597,TANJUNG PRIOK
1,MOBENG SUNTER - Ganti Oli & Servis Mobil,car_repair,-6.153459,106.879433,"Jl Sunter Indah Raya KA-1 No.24, RW.12, Sunter...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.5,,117,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",0812-9069-9681,ChIJVyKleIb1aS4ROLCg1Rbf4nw,,https://maps.google.com/?cid=8999000294606024760,TANJUNG PRIOK
2,Astra Otoservice,car_repair,-6.161987,106.904155,"Jl. Boulevard Raya Blok WE2 No.1, Jl. Boulevar...",OPERATIONAL,"[car_repair, auto_parts_store, point_of_intere...",,4.9,,761,"[{'open': {'day': 0, 'hour': 8, 'minute': 0}, ...",0819-9150-0015,ChIJIcZWPcL1aS4RWCa3o67kKek,https://instagram.com/astraotoservice?igshid=N...,https://maps.google.com/?cid=16801211323604477528,TANJUNG PRIOK
3,BUTIK OLI MOBIL,car_repair,-6.146281,106.896974,"Jl. Boulevard Artha Gading Blok A7A No.1, RT.1...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.9,,679,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",0821-7217-5858,ChIJ3eBr-kT1aS4RfMFk1F1Ypis,,https://maps.google.com/?cid=3145298549783445884,TANJUNG PRIOK
4,Super Shop & Drive Kelapa Gading,car_repair,-6.155265,106.896744,"Jl. Boulevard Bar. Raya No.1 blok LC 8, patoka...",OPERATIONAL,"[car_repair, auto_parts_store, point_of_intere...",,4.8,,993,"[{'open': {'day': 0, 'hour': 0, 'minute': 0}}]",(021) 45854111,ChIJrx3_tlf1aS4RN7F2447W1b4,http://wa.me/622145854111,https://maps.google.com/?cid=13751132946435584311,TANJUNG PRIOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,EJM Motor,car_repair,-6.128575,106.794760,"VQCV+HWC, Jl. Pluit Dalam III, RT.15/RW.8, Pen...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,1,"[{'open': {'day': 0, 'hour': 10, 'minute': 0},...",,ChIJz_jqdVodai4Rpmlb3WDEmcI,,https://maps.google.com/?cid=14022454835104410022,PENJARINGAN
416,Man Motor,car_repair,-6.165446,106.749473,"Jl. Basmol Raya No.146, RT.8/RW.4, Kembangan U...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,2,"[{'open': {'day': 0, 'hour': 10, 'minute': 0},...",0857-8089-9069,ChIJi4kprr33aS4RvGXnLlrWlHc,,https://maps.google.com/?cid=8616747669888918972,PENJARINGAN
417,Fajar Motor,car_repair,-6.134666,106.781800,"Jl. K, Tlk. Gong, No.35, RT.10/RW.10, Pejagala...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,1,"[{'open': {'day': 0, 'hour': 11, 'minute': 0},...",0852-1051-1188,ChIJO_Je2ocdai4RzP6rcL6rdpQ,,https://maps.google.com/?cid=10697926799287647948,PENJARINGAN
418,Tambal ban Libra oil,car_repair,-6.136940,106.756251,"15, Jl. Pos Polisi No.30, RT.2/RW.5, Kapuk, Ke...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,,,,"[{'open': {'day': 0, 'hour': 6, 'minute': 0}, ...",,ChIJD3Ew73gdai4R58c8-mJIyXo,,https://maps.google.com/?cid=8847682532888332263,PENJARINGAN


## Auto Parts

In [ ]:
# CONFIGURATION
radius = 3000 # in meters
keyword = 'auto parts' # input keyword
max_result = 20

# Headers untuk permintaan API
params = [
    'places.id', 'places.accessibilityOptions', 'places.addressComponents', 'places.adrFormatAddress',
    'places.businessStatus', 'places.displayName', 'places.formattedAddress', 'places.googleMapsUri',
    'places.iconBackgroundColor', 'places.iconMaskBaseUri', 'places.location', 'places.photos',
    'places.plusCode', 'places.primaryType', 'places.primaryTypeDisplayName', 'places.shortFormattedAddress',
    'places.subDestinations', 'places.types', 'places.utcOffsetMinutes', 'places.viewport', 'places.currentOpeningHours',
    'places.currentSecondaryOpeningHours', 'places.internationalPhoneNumber', 'places.nationalPhoneNumber', 'places.priceLevel',
    'places.rating', 'places.regularOpeningHours', 'places.regularSecondaryOpeningHours', 'places.userRatingCount', 'places.websiteUri'
]

headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': ','.join(params) + ',nextPageToken'
}

In [ ]:
# Loop melalui setiap baris dalam DataFrame dan panggil Text Search API
nearby_auto_parts = []
for index, row in df.iterrows():
    latitude = float(row['latitude'])
    longitude = float(row['longitude'])
    place_name = row['place']
    places = get_all_places(keyword, max_result, latitude, longitude, place_name)
    nearby_auto_parts.extend(places)

Requesting page 1 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 2 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
Requesting page 3 for TANJUNG PRIOK
Received 20 places for TANJUNG PRIOK
No more results or next page token found for TANJUNG PRIOK
Requesting page 1 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 2 for PADEMANGAN
Received 20 places for PADEMANGAN
Requesting page 3 for PADEMANGAN
Received 20 places for PADEMANGAN
No more results or next page token found for PADEMANGAN
Requesting page 1 for CILINCING
Received 20 places for CILINCING
Requesting page 2 for CILINCING
Received 20 places for CILINCING
Requesting page 3 for CILINCING
Received 20 places for CILINCING
No more results or next page token found for CILINCING
Requesting page 1 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 2 for KELAPA GADING
Received 20 places for KELAPA GADING
Requesting page 3 for KELAPA GADING
Received 20 places for KELAPA GAD

In [ ]:
# Saving JSON
with open('nearby_auto_parts.json', 'w') as f: # Change name as POI
    json.dump(nearby_auto_parts, f, indent=4)  # Change name as POI

In [ ]:
def extract_POI(data):
    extract_auto_parts = [] # Change name as POI
    for item in data:
        extract = {}
        extract['name'] = item.get('displayName', {}).get('text', '')
        extract['primary_type'] = item.get('primaryType', '')
        extract['latitude'] = item.get('location', {}).get('latitude', '')
        extract['longitude'] = item.get('location', {}).get('longitude', '')
        extract['address'] = item.get('formattedAddress', '')
        extract['business_status'] = item.get('businessStatus', '')
        extract['types'] = item.get('types', '')
        extract['price_level'] = item.get('priceLevel', '')
        extract['rating'] = item.get('rating', '')
        extract['reviews'] = item.get('reviews', '')
        extract['user_ratings_total'] = item.get('userRatingCount', '')
        extract['operational_hours'] = item.get('regularOpeningHours', {}).get('periods', '')
        extract['phone_number'] = item.get('nationalPhoneNumber', '')
        extract['place_id'] = item.get('id', '')
        extract['website_url'] = item.get('websiteUri', '')
        extract['gmaps_url'] = item.get('googleMapsUri', '')
        extract['origin_place'] = item.get('origin_place', '')
        # extract['wheelchair_parking'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleParking', '')
        # extract['wheelchair_entrance'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleEntrance', '')
        # extract['wheelchair_restroom'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleRestroom', '')
        # extract['wheelchair_seating'] = item.get('accessibilityOptions', {}).get('wheelchairAccessibleSeating', '')
        # extract['allow_dogs'] = item.get('allowsDogs', '')
        # extract['curbside_pickup'] = item.get('curbsidePickup', '')
        # extract['delivery'] = item.get('delivery', '')
        # extract['dine_in'] = item.get('dineIn', '')
        # extract['good_for_children'] = item.get('goodForChildren', '')
        # extract['good_for_groups'] = item.get('goodForGroups', '')
        # extract['good_for_watching_sports'] = item.get('goodForWatchingSports', '')
        # extract['live_music'] = item.get('liveMusic', '')
        # extract['menu_for_children'] = item.get('menuForChildren', '')
        # extract['paid_parking_lot'] = item.get('parkingOptions', {}).get('paidParkingLot', '')
        # extract['paid_street_parking'] = item.get('parkingOptions', {}).get('paidStreetParking', '')
        # extract['valet_parking'] = item.get('parkingOptions', {}).get('valetParking', '')
        # extract['credit_card_payment'] = item.get('paymentOptions', {}).get('acceptsCreditCards', '')
        # extract['debit_card_payment'] = item.get('paymentOptions', {}).get('acceptsDebitCards', '')
        # extract['only_cash_payment'] = item.get('paymentOptions', {}).get('acceptsCashOnly', '')
        # extract['nfc_payment'] = item.get('paymentOptions', {}).get('acceptsNfc', '')
        # extract['online_payment'] = item.get('paymentOptions', {}).get('acceptsOnlinePayments', '')
        # extract['outdoor_seating'] = item.get('outdoorSeating', '')
        # extract['reservable'] = item.get('reservable', '')
        # extract['restroom'] = item.get('restroom', '')
        # extract['serves_beer'] = item.get('servesBeer', '')
        # extract['serves_breakfast'] = item.get('servesBreakfast', '')
        # extract['serves_brunch'] = item.get('servesBrunch', '')
        # extract['serves_cocktails'] = item.get('servesCocktails', '')
        # extract['serves_coffee'] = item.get('servesCoffee', '')
        # extract['serves_dessert'] = item.get('servesDessert', '')
        # extract['serves_dinner'] = item.get('servesDinner', '')
        # extract['serves_lunch'] = item.get('servesLunch', '')
        # extract['serves_vegetarian_food'] = item.get('servesVegetarianFood', '')
        # extract['serves_wine'] = item.get('servesWine', '')
        # extract['takeout'] = item.get('takeout', '')
        extract_auto_parts.append(extract)  # Change name as POI
    return extract_auto_parts  # Change name as POI

# Flatten the JSON data
extract_auto_parts = extract_POI(nearby_auto_parts) # change name as POI

# Convert the flattened data to a DataFrame
POI_auto_parts = pd.DataFrame(extract_auto_parts) # Change name as POI
POI_auto_parts.to_csv('POI_auto_parts.csv', index=False)

In [ ]:
POI_auto_parts

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place
0,COD Autoparts,auto_parts_store,-6.151408,106.868556,"Jl. Bentengan II No.24 10, RT.2/RW.5, Sunter J...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,"[{'open': {'day': 0, 'hour': 9, 'minute': 0}, ...",0818-8286-3435,ChIJj-jgKmb1aS4RNsMTT87FOUc,,https://maps.google.com/?cid=5132350740246020918,TANJUNG PRIOK
1,Bengkel Matic Auto Garage,auto_parts_store,-6.152654,106.846634,"Ps. Mobil Kemayoran, Jl. Industri Raya No.10, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,4.8,,43,"[{'open': {'day': 1, 'hour': 10, 'minute': 0},...",0822-1160-6077,ChIJ____JB0eai4Rug5YsYfBgGg,,https://maps.google.com/?cid=7530231365503553210,TANJUNG PRIOK
2,Cahaya Auto Part,auto_parts_store,-6.157938,106.824937,"Jl. Taman Sari VI No.25B, RT.14/RW.7, Maphar, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,4.7,,32,"[{'open': {'day': 1, 'hour': 9, 'minute': 0}, ...",,ChIJ_ThXkRv1aS4RmqRsfyjnpG8,,https://maps.google.com/?cid=8044808995513083034,TANJUNG PRIOK
3,Astra Peugeot National Parts Center,car_repair,-6.132553,106.887827,"Komplek ex PPL, Jl. Gaya Motor II No.1, RT.9/R...",OPERATIONAL,"[market, auto_parts_store, car_repair, store, ...",,4.8,,4,"[{'open': {'day': 1, 'hour': 7, 'minute': 30},...",(021) 6510268,ChIJk0nFenj1aS4RWRX1uIrOoVI,http://www.peugeot.co.id/,https://maps.google.com/?cid=5954267277564384601,TANJUNG PRIOK
4,Mentari Autoparts,auto_parts_store,-6.109704,106.889248,"Jl. Enggano No.15, Tj. Priok, Kec. Tj. Priok, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,3.5,,2,"[{'open': {'day': 1, 'hour': 8, 'minute': 30},...",(021) 43909082,ChIJAzxDjpUfai4RqrcyieCz4wU,http://www.mentariautoparts.com/car_parts.html,https://maps.google.com/?cid=424380566858807210,TANJUNG PRIOK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,PT. Lancar Buana Sejati,auto_parts_store,-6.135664,106.727850,Jl. Taman Palem Lestari Jl. Ruko Pelangi No.30...,OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,"[{'open': {'day': 1, 'hour': 9, 'minute': 0}, ...",0818-168-986,ChIJKz1D0sgdai4R8ZQqTX5ROYA,https://www.lancarbuanasejati.com/,https://maps.google.com/?cid=9239505713429648625,PENJARINGAN
416,Takayama Motorcycle Parts,auto_parts_store,-6.123272,106.733795,"Kamal Muara, Penjaringan, North Jakarta City, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,,,,,,ChIJByugSk8dai4RUTZnbQBxE6o,,https://maps.google.com/?cid=12255263257654670929,PENJARINGAN
417,Leo Spare Parts,auto_parts_store,-6.118813,106.710904,"Jl. Manyar Raya No.3, RT.3/RW.11, Tegal Alur, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,,,ChIJ4yv698cCai4Rmpt1RuIXUU8,,https://maps.google.com/?cid=5715375662722030490,PENJARINGAN
418,Kita Carbon,auto_parts_store,-6.127402,106.727111,"Jl. Kencana Raya No.23 Blok C1, RT.1/RW.12, Ce...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,,,,"[{'open': {'day': 1, 'hour': 10, 'minute': 30}...",0851-7417-7128,ChIJfW51iPEdai4RxITkHiuy7cc,http://bodykitindonesia.com/,https://maps.google.com/?cid=14406366681299715268,PENJARINGAN


# APPEND RESULT

In [ ]:
keyword_list = [
    'bengkel_mobil', 'bengkel_motor', 'dealer', 'bengkel_oli', 'auto_parts'
]

dataframes = []
for keyword in keyword_list:
    dataframe_name = f"POI_{keyword}"
    df = globals()[dataframe_name]
    df['keyword'] = keyword
    dataframes.append(df)

All_POI = pd.concat(dataframes, ignore_index=True)

In [ ]:
All_POI

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place,keyword
0,Jaya Bengkel,car_repair,-6.127449,106.873610,"Papanggo, Tanjung Priok, North Jakarta City, J...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,,,,,,ChIJwW3dHtEfai4RRxCbe0s70wk,,https://maps.google.com/?cid=707974761809645639,TANJUNG PRIOK,bengkel_mobil
1,Restu Jaya Motor,car_repair,-6.132836,106.871622,Jl Nusantara Timur Sunter Agung Tanjung Priok ...,OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.5,,45,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",(021) 6505373,ChIJZ_H6nXn1aS4RPA-fSjNDlas,,https://maps.google.com/?cid=12363862239589895996,TANJUNG PRIOK,bengkel_mobil
2,Bengkel Mobil Bang Jun,car_repair,-6.127169,106.875933,"Jl. Papanggo IV Rw05 No.11, RT.5/RW.5, Papangg...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,2,,0899-9004-035,ChIJa52PRw4fai4RMElGbpbwhoI,,https://maps.google.com/?cid=9405469400695720240,TANJUNG PRIOK,bengkel_mobil
3,Bengkel Mobil Fortuna Motor,car_repair,-6.127462,106.893272,"Jl. Plumpang Semper raya No.5 12, RT.15/RW.4, ...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.8,,48,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",0858-1437-9748,ChIJ20KI5sIfai4RpwqGL9-1b_4,,https://maps.google.com/?cid=18334072578228226727,TANJUNG PRIOK,bengkel_mobil
4,Bengkel Mobil Cars City,car_repair,-6.138704,106.866383,"Jl. Danau Sunter Utara No.13 Blok F 20, RT.11/...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.8,,6,"[{'open': {'day': 1, 'hour': 9, 'minute': 0}, ...",0813-8081-9995,ChIJUYRyRn_1aS4RFseZAD39er8,,https://maps.google.com/?cid=13797618846847846166,TANJUNG PRIOK,bengkel_mobil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,PT. Lancar Buana Sejati,auto_parts_store,-6.135664,106.727850,Jl. Taman Palem Lestari Jl. Ruko Pelangi No.30...,OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,"[{'open': {'day': 1, 'hour': 9, 'minute': 0}, ...",0818-168-986,ChIJKz1D0sgdai4R8ZQqTX5ROYA,https://www.lancarbuanasejati.com/,https://maps.google.com/?cid=9239505713429648625,PENJARINGAN,auto_parts
2096,Takayama Motorcycle Parts,auto_parts_store,-6.123272,106.733795,"Kamal Muara, Penjaringan, North Jakarta City, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,,,,,,ChIJByugSk8dai4RUTZnbQBxE6o,,https://maps.google.com/?cid=12255263257654670929,PENJARINGAN,auto_parts
2097,Leo Spare Parts,auto_parts_store,-6.118813,106.710904,"Jl. Manyar Raya No.3, RT.3/RW.11, Tegal Alur, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,,,ChIJ4yv698cCai4Rmpt1RuIXUU8,,https://maps.google.com/?cid=5715375662722030490,PENJARINGAN,auto_parts
2098,Kita Carbon,auto_parts_store,-6.127402,106.727111,"Jl. Kencana Raya No.23 Blok C1, RT.1/RW.12, Ce...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,,,,"[{'open': {'day': 1, 'hour': 10, 'minute': 30}...",0851-7417-7128,ChIJfW51iPEdai4RxITkHiuy7cc,http://bodykitindonesia.com/,https://maps.google.com/?cid=14406366681299715268,PENJARINGAN,auto_parts


In [ ]:
def format_open_hours(hours_json):
    if hours_json is None or (isinstance(hours_json, float) and pd.isna(hours_json)) or hours_json == "":
        return "Not Available"

    try:
        # Jika input adalah string, parse sebagai JSON
        if isinstance(hours_json, str):
            hours_list = json.loads(hours_json.replace("'", '"'))
        else:
            hours_list = hours_json

        # Dictionary to convert day index to day name
        day_names = {0: 'Sunday', 1: 'Monday', 2: 'Tuesday', 3: 'Wednesday',
                     4: 'Thursday', 5: 'Friday', 6: 'Saturday'}
        # Initialize result string
        formatted_hours = ""
        for item in hours_list:
            try:
                day = item['open']['day']
                open_time = f"{item['open']['hour']:02}:{item['open']['minute']:02}"
                close_time = f"{item['close']['hour']:02}:{item['close']['minute']:02}" if 'close' in item else "00:00"
                if day in day_names:
                    formatted_hours += f"[{day_names[day]}: {open_time} - {close_time}], "
            except KeyError as ke:
                print(f"KeyError in item: {item}\nException: {ke}")
                return "Not Available"
        return formatted_hours.strip(', ')  # Remove the last comma
    except json.JSONDecodeError as jde:
        print(f"JSONDecodeError: {hours_json}\nException: {jde}")
        return "Not Available"
    except Exception as e:
        print(f"Error processing hours_json: {hours_json}\nException: {e}")
        return "Not Available"

In [ ]:
All_POI['open_hour'] = All_POI['operational_hours'].apply(format_open_hours)

In [ ]:
All_POI[['name', 'open_hour']].head()

,name,open_hour
0,Jaya Bengkel,Not Available
1,Restu Jaya Motor,"[Monday: 08:00 - 16:30], [Tuesday: 08:00 - 16:..."
2,Bengkel Mobil Bang Jun,Not Available
3,Bengkel Mobil Fortuna Motor,"[Monday: 08:00 - 17:00], [Tuesday: 08:00 - 17:..."
4,Bengkel Mobil Cars City,"[Monday: 09:00 - 17:00], [Tuesday: 09:00 - 17:..."


In [ ]:
##BNI_POI = BNI_POI['open_hour'].drop

In [ ]:
All_POI.to_csv('All_POI_dup.csv', index=False)

In [ ]:
All_POI_clean = All_POI.drop_duplicates(subset='place_id', keep='first', inplace=False)

In [ ]:
All_POI_clean

,name,primary_type,latitude,longitude,address,business_status,types,price_level,rating,reviews,user_ratings_total,operational_hours,phone_number,place_id,website_url,gmaps_url,origin_place,keyword,open_hour
0,Jaya Bengkel,car_repair,-6.127449,106.873610,"Papanggo, Tanjung Priok, North Jakarta City, J...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,,,,,,ChIJwW3dHtEfai4RRxCbe0s70wk,,https://maps.google.com/?cid=707974761809645639,TANJUNG PRIOK,bengkel_mobil,Not Available
1,Restu Jaya Motor,car_repair,-6.132836,106.871622,Jl Nusantara Timur Sunter Agung Tanjung Priok ...,OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.5,,45,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",(021) 6505373,ChIJZ_H6nXn1aS4RPA-fSjNDlas,,https://maps.google.com/?cid=12363862239589895996,TANJUNG PRIOK,bengkel_mobil,"[Monday: 08:00 - 16:30], [Tuesday: 08:00 - 16:..."
2,Bengkel Mobil Bang Jun,car_repair,-6.127169,106.875933,"Jl. Papanggo IV Rw05 No.11, RT.5/RW.5, Papangg...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,5,,2,,0899-9004-035,ChIJa52PRw4fai4RMElGbpbwhoI,,https://maps.google.com/?cid=9405469400695720240,TANJUNG PRIOK,bengkel_mobil,Not Available
3,Bengkel Mobil Fortuna Motor,car_repair,-6.127462,106.893272,"Jl. Plumpang Semper raya No.5 12, RT.15/RW.4, ...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.8,,48,"[{'open': {'day': 1, 'hour': 8, 'minute': 0}, ...",0858-1437-9748,ChIJ20KI5sIfai4RpwqGL9-1b_4,,https://maps.google.com/?cid=18334072578228226727,TANJUNG PRIOK,bengkel_mobil,"[Monday: 08:00 - 17:00], [Tuesday: 08:00 - 17:..."
4,Bengkel Mobil Cars City,car_repair,-6.138704,106.866383,"Jl. Danau Sunter Utara No.13 Blok F 20, RT.11/...",OPERATIONAL,"[car_repair, point_of_interest, establishment]",,4.8,,6,"[{'open': {'day': 1, 'hour': 9, 'minute': 0}, ...",0813-8081-9995,ChIJUYRyRn_1aS4RFseZAD39er8,,https://maps.google.com/?cid=13797618846847846166,TANJUNG PRIOK,bengkel_mobil,"[Monday: 09:00 - 17:00], [Tuesday: 09:00 - 17:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,PT. Lancar Buana Sejati,auto_parts_store,-6.135664,106.727850,Jl. Taman Palem Lestari Jl. Ruko Pelangi No.30...,OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,"[{'open': {'day': 1, 'hour': 9, 'minute': 0}, ...",0818-168-986,ChIJKz1D0sgdai4R8ZQqTX5ROYA,https://www.lancarbuanasejati.com/,https://maps.google.com/?cid=9239505713429648625,PENJARINGAN,auto_parts,"[Monday: 09:00 - 17:00], [Tuesday: 09:00 - 17:..."
2096,Takayama Motorcycle Parts,auto_parts_store,-6.123272,106.733795,"Kamal Muara, Penjaringan, North Jakarta City, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,,,,,,ChIJByugSk8dai4RUTZnbQBxE6o,,https://maps.google.com/?cid=12255263257654670929,PENJARINGAN,auto_parts,Not Available
2097,Leo Spare Parts,auto_parts_store,-6.118813,106.710904,"Jl. Manyar Raya No.3, RT.3/RW.11, Tegal Alur, ...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,5,,1,,,ChIJ4yv698cCai4Rmpt1RuIXUU8,,https://maps.google.com/?cid=5715375662722030490,PENJARINGAN,auto_parts,Not Available
2098,Kita Carbon,auto_parts_store,-6.127402,106.727111,"Jl. Kencana Raya No.23 Blok C1, RT.1/RW.12, Ce...",OPERATIONAL,"[auto_parts_store, car_repair, store, point_of...",,,,,"[{'open': {'day': 1, 'hour': 10, 'minute': 30}...",0851-7417-7128,ChIJfW51iPEdai4RxITkHiuy7cc,http://bodykitindonesia.com/,https://maps.google.com/?cid=14406366681299715268,PENJARINGAN,auto_parts,"[Monday: 10:30 - 17:00], [Tuesday: 10:30 - 17:..."


In [ ]:
All_POI_clean.to_csv('All_POI_clean.csv', index=False)